# Importing Libraries

In [80]:
import pandas as pd
import nltk
import re
from torchvision.datasets import ImageFolder
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
from sklearn.model_selection import train_test_split
from torch import tensor
import torchvision
import torchvision.transforms.functional as TF
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import torch.optim as optim
import numpy as np
import ast
from torch.utils.data import TensorDataset, DataLoader


In [9]:
os.chdir("..")

In [10]:
df=pd.read_csv("sentiment/sentiment.csv")
df

Unnamed: 0  imgid  split                       filename  successful  \
0               0  31369  train  COCO_val2014_000000389081.jpg           1   
1               1  31369  train  COCO_val2014_000000389081.jpg           1   
2               2  31369  train  COCO_val2014_000000389081.jpg           1   
3               3  31369  train  COCO_val2014_000000389081.jpg           1   
4               4  31369  train  COCO_val2014_000000389081.jpg           1   
...           ...    ...    ...                            ...         ...   
39194       39194  24628   test  COCO_val2014_000000190705.jpg           1   
39195       39195  24628   test  COCO_val2014_000000190705.jpg           1   
39196       39196  24628   test  COCO_val2014_000000190705.jpg           1   
39197       39197  24628   test  COCO_val2014_000000190705.jpg           1   
39198       39198  24628   test  COCO_val2014_000000190705.jpg           1   

                                                  tokens  \
0      ['a', 'plate', 'of', 'delicious', 'food', 'inc...   
1      ['french', 'fries', 'are', 'not', 'a', 'health...   
2      ['the', 'plate', 'has', 'one', 'of', 'my', 'fa...   
3      ['it', 'was', 'disgusting', 'food', 'not', 'ju...   
4      ['a', 'plate', 'of', 'disgusting', 'food', 'fo...   
...                                                  ...   
39194  ['a', 'dirty', 'bathroom', 'that', 'has', 'a',...   
39195  ['a', 'dirty', 'bathroom', 'that', 'has', 'a',...   
39196  ['a', 'towel', 'that', 'is', 'on', 'a', 'rack'...   
39197  ['a', 'dirty', 'bathroom', 'that', 'has', 'a',...   
39198  ['a', 'dirty', 'bathroom', 'that', 'has', 'a',...   

                                          word_sentiment  sentiment  \
0                   [0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0]          1   
1      [0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0,...          1   
2      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0,...          1   
3                       [0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]          0   
4              [0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0]          0   
...                                                  ...        ...   
39194   [0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]          0   
39195            [0.0, 1, 1, 0.0, 0.0, 0.0, 1, 0.0, 0.0]          0   
39196  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...          0   
39197   [0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]          0   
39198            [0.0, 1, 1, 0.0, 0.0, 0.0, 1, 0.0, 0.0]          0   

                                                     raw  
0      a plate of delicious food including French fries.  
1      French fries are not a healthy food but it is ...  
2      The plate has one of my favorite foods on it, ...  
3             It was disgusting food, not just bad food.  
4           A plate of disgusting food found at a diner.  
...                                                  ...  
39194  A dirty bathroom that has a dirty window made ...  
39195          A dirty bathroom that has a window in it.  
39196      a towel that is on a rack in a dirty bathroom  
39197  A dirty bathroom that has a dirty window made ...  
39198          A dirty bathroom that has a window in it.  

[39199 rows x 9 columns]

In [11]:
def clean(x):
    return nltk.word_tokenize(re.sub(r'[^\w\s]', '', x.lower()))
df["raw"]=df['raw'].apply(clean)
df["raw"]

0        [a, plate, of, delicious, food, including, fre...
1        [french, fries, are, not, a, healthy, food, bu...
2        [the, plate, has, one, of, my, favorite, foods...
3        [it, was, disgusting, food, not, just, bad, food]
4        [a, plate, of, disgusting, food, found, at, a,...
                               ...                        
39194    [a, dirty, bathroom, that, has, a, dirty, wind...
39195    [a, dirty, bathroom, that, has, a, window, in,...
39196    [a, towel, that, is, on, a, rack, in, a, dirty...
39197    [a, dirty, bathroom, that, has, a, dirty, wind...
39198    [a, dirty, bathroom, that, has, a, window, in,...
Name: raw, Length: 39199, dtype: object

In [12]:
import torch
from transformers import BertTokenizer
import pandas as pd

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def encode_sentences(sentences):
    tokenized = [tokenizer.encode(s, add_special_tokens=True) for s in sentences]
    max_len = max([len(x) for x in tokenized])
    padded = [x + [0] * (max_len - len(x)) for x in tokenized]
    input_ids = torch.tensor(padded)
    attention_masks = torch.where(input_ids != 0, torch.tensor(1), torch.tensor(0))
    return input_ids, attention_masks

# Encode sentences from the 'raw' column
input_ids, attention_masks = encode_sentences(df["raw"])

print("Input IDs shape:", input_ids.shape)

my_array = input_ids.numpy()
df["en"] = pd.DataFrame({'tensor_col': [torch.from_numpy(my_array[i]) for i in range(my_array.shape[0])]})

print(df["en"])


Input IDs shape: torch.Size([39199, 98])
0        [tensor(101), tensor(1037), tensor(5127), tens...
1        [tensor(101), tensor(2413), tensor(22201), ten...
2        [tensor(101), tensor(1996), tensor(5127), tens...
3        [tensor(101), tensor(2009), tensor(2001), tens...
4        [tensor(101), tensor(1037), tensor(5127), tens...
                               ...                        
39194    [tensor(101), tensor(1037), tensor(6530), tens...
39195    [tensor(101), tensor(1037), tensor(6530), tens...
39196    [tensor(101), tensor(1037), tensor(10257), ten...
39197    [tensor(101), tensor(1037), tensor(6530), tens...
39198    [tensor(101), tensor(1037), tensor(6530), tens...
Name: en, Length: 39199, dtype: object


In [13]:
import numpy as np
import pandas as pd

# Convert tensors to NumPy arrays of float32
df['en'] = df['en'].apply(lambda x: np.array(x.numpy()).astype('float32'))

# Print the modified 'en' column
print(df['en'])

0        [101.0, 1037.0, 5127.0, 1997.0, 12090.0, 2833....
1        [101.0, 2413.0, 22201.0, 2024.0, 2025.0, 1037....
2        [101.0, 1996.0, 5127.0, 2038.0, 2028.0, 1997.0...
3        [101.0, 2009.0, 2001.0, 19424.0, 2833.0, 2025....
4        [101.0, 1037.0, 5127.0, 1997.0, 19424.0, 2833....
                               ...                        
39194    [101.0, 1037.0, 6530.0, 5723.0, 2008.0, 2038.0...
39195    [101.0, 1037.0, 6530.0, 5723.0, 2008.0, 2038.0...
39196    [101.0, 1037.0, 10257.0, 2008.0, 2003.0, 2006....
39197    [101.0, 1037.0, 6530.0, 5723.0, 2008.0, 2038.0...
39198    [101.0, 1037.0, 6530.0, 5723.0, 2008.0, 2038.0...
Name: en, Length: 39199, dtype: object


In [14]:
my_list = df["word_sentiment"].to_list()
max_len = max([len(l) for l in my_list])

def pad(x):
    x_list = ast.literal_eval(x)
    return np.pad(x_list, (0, max_len - len(x_list)), mode='constant')

df["word_sentiment"] = df['word_sentiment'].apply(pad)

print(df["word_sentiment"])

0        [0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...
1        [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...
2        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...
3        [0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...
4        [0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...
                               ...                        
39194    [0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...
39195    [0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...
39196    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
39197    [0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...
39198    [0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...
Name: word_sentiment, Length: 39199, dtype: object


# Image Resizing

In [15]:
import os
from PIL import Image
os.chdir("sentiment")

In [16]:
os.chdir("sentiment_images")

In [17]:
files = [f for f in os.listdir(os.getcwd()) if f]

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

resnet = torchvision.models.resnet18(pretrained=True)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])
resnet.eval()

images = []
labels = []

for filename in files:
    try:
        img = Image.open(filename)
        img = transform(img)
        if img.shape[0] == 1:
            img = img.expand(3, -1, -1)
        elif img.shape[0] != 3:
            raise ValueError("Unsupported image format: %s" % filename)
        img = img.unsqueeze(0)
        features = resnet(img)
        features = features.squeeze().detach().numpy()
        images.append(features)
        labels.append(filename)
    except Exception as e:
        print("Error processing image:", filename)
        print(e)

images = np.array(images)
labels = np.array(labels)

print("Images shape:", images.shape)
print("Labels shape:", labels.shape)


C:\Users\USER\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Images shape: (2221, 512)
Labels shape: (2221,)


In [18]:
os.chdir("..")

In [19]:
image_features = []
notPresent = []

for filename in df['filename']:
    found = False
    for i in range(len(labels)):
        if labels[i] == filename:
            image_features.append(images[i])
            found = True
            break
    if not found:
        if filename not in notPresent:
            notPresent.append(filename)


In [20]:
df = df[~df['filename'].isin(notPresent)]

In [21]:
df["image_features"]=image_features

C:\Users\USER\AppData\Local\Temp/ipykernel_2976/91944046.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["image_features"]=image_features


# Train/Test Split: Done according to the given requirement

In [22]:
df=df.reset_index(drop=True)
df

Unnamed: 0  imgid  split                       filename  successful  \
0               0  31369  train  COCO_val2014_000000389081.jpg           1   
1               1  31369  train  COCO_val2014_000000389081.jpg           1   
2               2  31369  train  COCO_val2014_000000389081.jpg           1   
3               3  31369  train  COCO_val2014_000000389081.jpg           1   
4               4  31369  train  COCO_val2014_000000389081.jpg           1   
...           ...    ...    ...                            ...         ...   
39104       39194  24628   test  COCO_val2014_000000190705.jpg           1   
39105       39195  24628   test  COCO_val2014_000000190705.jpg           1   
39106       39196  24628   test  COCO_val2014_000000190705.jpg           1   
39107       39197  24628   test  COCO_val2014_000000190705.jpg           1   
39108       39198  24628   test  COCO_val2014_000000190705.jpg           1   

                                                  tokens  \
0      ['a', 'plate', 'of', 'delicious', 'food', 'inc...   
1      ['french', 'fries', 'are', 'not', 'a', 'health...   
2      ['the', 'plate', 'has', 'one', 'of', 'my', 'fa...   
3      ['it', 'was', 'disgusting', 'food', 'not', 'ju...   
4      ['a', 'plate', 'of', 'disgusting', 'food', 'fo...   
...                                                  ...   
39104  ['a', 'dirty', 'bathroom', 'that', 'has', 'a',...   
39105  ['a', 'dirty', 'bathroom', 'that', 'has', 'a',...   
39106  ['a', 'towel', 'that', 'is', 'on', 'a', 'rack'...   
39107  ['a', 'dirty', 'bathroom', 'that', 'has', 'a',...   
39108  ['a', 'dirty', 'bathroom', 'that', 'has', 'a',...   

                                          word_sentiment  sentiment  \
0      [0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...          1   
1      [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...          1   
2      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...          1   
3      [0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...          0   
4      [0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...          0   
...                                                  ...        ...   
39104  [0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...          0   
39105  [0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...          0   
39106  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...          0   
39107  [0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...          0   
39108  [0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...          0   

                                                     raw  \
0      [a, plate, of, delicious, food, including, fre...   
1      [french, fries, are, not, a, healthy, food, bu...   
2      [the, plate, has, one, of, my, favorite, foods...   
3      [it, was, disgusting, food, not, just, bad, food]   
4      [a, plate, of, disgusting, food, found, at, a,...   
...                                                  ...   
39104  [a, dirty, bathroom, that, has, a, dirty, wind...   
39105  [a, dirty, bathroom, that, has, a, window, in,...   
39106  [a, towel, that, is, on, a, rack, in, a, dirty...   
39107  [a, dirty, bathroom, that, has, a, dirty, wind...   
39108  [a, dirty, bathroom, that, has, a, window, in,...   

                                                      en  \
0      [101.0, 1037.0, 5127.0, 1997.0, 12090.0, 2833....   
1      [101.0, 2413.0, 22201.0, 2024.0, 2025.0, 1037....   
2      [101.0, 1996.0, 5127.0, 2038.0, 2028.0, 1997.0...   
3      [101.0, 2009.0, 2001.0, 19424.0, 2833.0, 2025....   
4      [101.0, 1037.0, 5127.0, 1997.0, 19424.0, 2833....   
...                                                  ...   
39104  [101.0, 1037.0, 6530.0, 5723.0, 2008.0, 2038.0...   
39105  [101.0, 1037.0, 6530.0, 5723.0, 2008.0, 2038.0...   
39106  [101.0, 1037.0, 10257.0, 2008.0, 2003.0, 2006....   
39107  [101.0, 1037.0, 6530.0, 5723.0, 2008.0, 2038.0...   
39108  [101.0, 1037.0, 6530.0, 5723.0, 2008.0, 2038.0...   

                                          image_featur

In [23]:
train_caption = df[df["split"] == "train"]

train_caption_image = torch.tensor(train_caption["image_features"].tolist())
train_caption_word = torch.tensor(train_caption["en"].tolist())
train_caption_label = torch.tensor(train_caption["sentiment"].values)

test_caption = df[df["split"] == "test"]

test_caption_image = torch.tensor(test_caption["image_features"].tolist())
test_caption_word = torch.tensor(test_caption["en"].tolist())
test_caption_label = torch.tensor(test_caption["sentiment"].values)


C:\Users\USER\AppData\Local\Temp/ipykernel_2976/3110314269.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:248.)
  train_caption_image = torch.tensor(train_caption["image_features"].tolist())


In [24]:
print(test_caption_image.shape)
print(test_caption_word.shape)
print(train_caption_label.shape)

torch.Size([15912, 512])
torch.Size([15912, 98])
torch.Size([19307])


# Designing Model Architecture

In [68]:
import torch.nn as nn
import torch.nn.functional as F

class MultimodalModel(nn.Module):
    def __init__(self, num_classes, vocab_size, embedding_dim, hidden_size, num_layers):
        super(MultimodalModel, self).__init__()

        self.text_embedding = nn.Embedding(vocab_size, embedding_dim)
        self.text_lstm = nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers, batch_first=True)

        self.visual_fc = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, hidden_size),
            nn.ReLU()
        )

        self.fc1 = nn.Linear(hidden_size + hidden_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, num_classes)

        self.dropout1 = nn.Dropout(p=0.5)
        self.dropout2 = nn.Dropout(p=0.5)
        self.dropout3 = nn.Dropout(p=0.5)
        self.dropout4 = nn.Dropout(p=0.5)

        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(128)
        self.bn4 = nn.BatchNorm1d(64)

    def forward(self, text, image):
        text = self.text_embedding(text.long())
        _, (text_output, _) = self.text_lstm(text)
        text_output = text_output[-1]

        visual_output = self.visual_fc(image)

        combined = torch.cat((text_output, visual_output), dim=1)

        output = F.relu(self.fc1(combined))
        output = self.dropout1(output)
        output = self.bn1(output)
        output = F.relu(self.fc2(output))
        output = self.dropout2(output)
        output = self.bn2(output)
        output = F.relu(self.fc3(output))
        output = self.dropout3(output)
        output = self.bn3(output)
        output = F.relu(self.fc4(output))
        output = self.dropout4(output)
        output = self.bn4(output)
        output = self.fc5(output)

        return output


# Training the model

In [69]:
max(tensor1.max() for tensor1 in test_caption_word) + 1

tensor(29572.)

In [70]:
train_caption=df[df["split"]=="train"]
train_caption_word= torch.tensor(train_caption["en"].tolist())
test_caption=df[df["split"]=="test"]
test_caption_word= torch.tensor(test_caption["en"].tolist())

In [74]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
num_epochs = 20
batch_size = 50
learning_rate = 0.001

# Model configuration
model = MultimodalModel(num_classes=2, vocab_size=29572, embedding_dim=256, hidden_size=128, num_layers=2)
model.to(device)

# Define loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Data loaders
dataset = TensorDataset(train_caption_image, train_caption_word, train_caption_label)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Training loop
for epoch in range(num_epochs):
    model.train()
    train_acc = 0

    for batch_caption_image, batch_caption_word, batch_caption_label in dataloader:
        batch_caption_image = batch_caption_image.to(device)
        batch_caption_word = batch_caption_word.to(device)
        batch_caption_label = F.one_hot(batch_caption_label, num_classes=2).float().to(device)

        optimizer.zero_grad()
        output = model(batch_caption_word, batch_caption_image)
        loss = criterion(output, batch_caption_label)
        loss.backward()
        optimizer.step()

        # Calculate training accuracy
        predicted_labels = torch.round(torch.sigmoid(output)).long()
        train_acc += (predicted_labels == batch_caption_label.squeeze(1).long()).sum().item()

    train_accuracy = train_acc / len(train_caption)

    # Validation
    model.eval()
    # Perform validation and monitor performance

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Train Accuracy: {train_accuracy:.4f}")

# Save the trained model
torch.save(model.state_dict(), "model.pt")

Epoch 1/20, Loss: 0.7096, Train Accuracy: 1.0302
Epoch 2/20, Loss: 0.6696, Train Accuracy: 1.0535
Epoch 3/20, Loss: 0.6586, Train Accuracy: 1.0698
Epoch 4/20, Loss: 0.7253, Train Accuracy: 1.0723
Epoch 5/20, Loss: 0.6905, Train Accuracy: 1.0704
Epoch 6/20, Loss: 0.7051, Train Accuracy: 1.0712
Epoch 7/20, Loss: 0.6989, Train Accuracy: 1.0725
Epoch 8/20, Loss: 0.7552, Train Accuracy: 1.0875
Epoch 9/20, Loss: 0.6595, Train Accuracy: 1.0929
Epoch 10/20, Loss: 0.6850, Train Accuracy: 1.1029
Epoch 11/20, Loss: 0.7153, Train Accuracy: 1.1136
Epoch 12/20, Loss: 0.6891, Train Accuracy: 1.1194
Epoch 13/20, Loss: 0.8413, Train Accuracy: 1.1298
Epoch 14/20, Loss: 0.7367, Train Accuracy: 1.1446
Epoch 15/20, Loss: 0.6784, Train Accuracy: 1.1542
Epoch 16/20, Loss: 0.7297, Train Accuracy: 1.1511
Epoch 17/20, Loss: 0.7842, Train Accuracy: 1.1436
Epoch 18/20, Loss: 0.7012, Train Accuracy: 1.1646
Epoch 19/20, Loss: 0.6376, Train Accuracy: 1.1655
Epoch 20/20, Loss: 0.6185, Train Accuracy: 1.1452



# Model Evaluation

In [79]:
from sklearn.metrics import precision_recall_fscore_support

model = MultimodalModel(num_classes=2, vocab_size=29572, embedding_dim=256, hidden_size=128, num_layers=2)
model.load_state_dict(torch.load("model.pt"))

test_dataset = TensorDataset(test_caption_image, test_caption_word, torch.tensor(test_caption["sentiment"].values))
batch_size = 35
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model.eval()
correct = 0
total = 0
predictions = []

with torch.no_grad():
    for batch_images, batch_text, batch_labels in test_dataloader:
        output = model(batch_text, batch_images)
        predicted_labels = torch.argmax(output, dim=1)
        correct += (predicted_labels == batch_labels).sum().item()
        total += batch_labels.size(0)
        predictions.extend(predicted_labels.tolist())

accuracy = 100 * correct / total
print("Accuracy: {:.2f}%".format(accuracy))

precision, recall, f1_score, _ = precision_recall_fscore_support(test_caption["sentiment"].values, predictions, average='binary')
print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F1 score: {:.2f}".format(f1_score))


Accuracy: 54.75%
Precision: 0.55
Recall: 1.00
F1 score: 0.71
